app.py

In [0]:
import streamlit as st
from databricks import sql
from databricks.sdk.core import Config
import pandas as pd

st.set_page_config(layout="wide")

# establish databricks sdk cor config
cfg = Config()

# function to read table
def read_table(table_name: str, conn) -> pd.DataFrame:
    with conn.cursor() as cursor:
        cursor.execute(f"Select * from {table_name}")
        return cursor.fetchall_arrow().to_pandas()

# function to establish connection with sql warehouse
def get_connection(http_path):
    return sql.connect(
        server_hostname=cfg.host,
        http_path=http_path,
        credentials_provider=lambda: cfg.authenticate
    )

# path of sql warehouse
http_path_input = "############"

# establish connection of sql warehouse
conn = get_connection(http_path_input)

# save table to data frame using function
df = read_table("workspace.default.demo_product_texts", conn)

# organize app layout by tabs
tab1, tab2 = st.tabs(["Read Data", "Write Data"])

# view the data frame and filter
with tab1:
    col1, col2 = st.columns([1,4])
    with col1:
        # Filter by name
        names = df["name"].unique().tolist()
        name_filter = st.selectbox("Filter by Name", ["All"] + sorted(names))

        if name_filter != "All":
            df = df[df["name"] == name_filter]
    
    with col2:
        # Display dataframe
        st.dataframe(df)

# Edit the dataframe
with tab2: 

    TABLE_NAME = "workspace.default.demo_product_texts"
    PRIMARY_KEY = "id"

    original_display_df = df.copy()
    edited_df = st.data_editor(df, num_rows="dynamic")

    # Save changes
    if st.button("Save Changes"):
        updates_made = 0
        for idx, row in edited_df.iterrows():
            original_row = original_display_df.loc[idx]

            #check if anything actually changed in row
            if not row.equals(original_row):
                # build set clause
                set_clause = ", ".join([f"{col} = {repr(row[col])}" for col in row.index])
                update_sql = f"""
                    UPDATE {TABLE_NAME}
                    SET {set_clause}
                    WHERE {PRIMARY_KEY} = {repr(row['id'])}
                """

                with st.spinner("Saving Changes..."):
                    with conn.cursor() as cursor:
                        cursor.execute(update_sql)
                    updates_made += 1
        if updates_made > 0:
            st.success(f"Saved {updates_made} changes!")
        else:
            st.info("No changes detected. Nothing Updated")
